# From SQL to pandas

In [1]:
import numpy as np
import pandas as pd

The data set is in an Excel file and represents data on sales.

In [8]:
sales = pd.read_excel('data/sales-funnel.xlsx')

First, we explore the data.

In [9]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 8 columns):
Account     17 non-null int64
Name        17 non-null object
Rep         17 non-null object
Manager     17 non-null object
Product     17 non-null object
Quantity    17 non-null int64
Price       17 non-null int64
Status      17 non-null object
dtypes: int64(3), object(5)
memory usage: 1.1+ KB


In [10]:
sales

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won
5,218895,Kulas Inc,Daniel Hilton,Debra Henley,CPU,2,40000,pending
6,218895,Kulas Inc,Daniel Hilton,Debra Henley,Software,1,10000,presented
7,412290,Jerde-Hilpert,John Smith,Debra Henley,Maintenance,2,5000,pending
8,740150,Barton LLC,John Smith,Debra Henley,CPU,1,35000,declined
9,141962,Herman LLC,Cedric Moss,Fred Anderson,CPU,2,65000,won


For each sale, there is a unique identifier (`Account`), and the customer (`Name`) is recorded, as well the name of the sales person (`Rep`), her manager (`Manager`), the product, quantity and price. Finally, there is status information on the sale.

```
select distinct Rep from sales;
```

In [14]:
sales.Rep.unique()

array(['Craig Booker', 'Daniel Hilton', 'John Smith', 'Cedric Moss',
       'Wendy Yule'], dtype=object)

```
select Manager, Rep from sales group by Rep order by Manager;
```

In [23]:
for key, value in sales[['Manager', 'Rep']].drop_duplicates().groupby('Manager'):
    print(f'{key}:\n {value}')

Debra Henley:
         Manager            Rep
0  Debra Henley   Craig Booker
4  Debra Henley  Daniel Hilton
7  Debra Henley     John Smith
Fred Anderson:
           Manager          Rep
9   Fred Anderson  Cedric Moss
13  Fred Anderson   Wendy Yule


```
select Manager, count(distinct Rep) from sales group by Manager;
```

In [25]:
sales[['Manager', 'Rep']].drop_duplicates().groupby('Manager').count()

,Rep
Manager,
Debra Henley,3
Fred Anderson,2


```
select Rep, count(*) from sales where Status = 'won' group by Rep;
```

In [36]:
sales[['Rep', 'Status']].query('Status == "won"').groupby('Rep').count()

,Status
Rep,
Cedric Moss,1
Daniel Hilton,1
Wendy Yule,2


```
select Manager, count(Status) from sales where Status = 'won' group by Manager;
```

In [29]:
sales[['Manager', 'Status']].query('Status == "won"').groupby('Manager').count()

,Status
Manager,
Debra Henley,1
Fred Anderson,3


In [39]:
sales[['Manager', 'Status']].query('Status == "won"').groupby('Manager').size()\
                            .to_frame('Won').sort_values('Won', ascending=False)

,Won
Manager,
Fred Anderson,3
Debra Henley,1


```
select Rep, Status, count(*) from sales group by Rep, Status;
```

In [53]:
sales[['Rep', 'Status']].groupby(['Rep', 'Status']).size().to_frame('Count')

Count
Rep           Status          
Cedric Moss   pending        1
              presented      2
              won            1
Craig Booker  declined       1
              pending        1
              presented      2
Daniel Hilton pending        1
              presented      1
              won            1
John Smith    declined       1
              pending        1
Wendy Yule    declined       1
              presented      1
              won            2

```
select Rep, Status, count(*) from sales group by Rep, Status;
```

In [52]:
sales[['Rep', 'Status']].groupby(['Rep', 'Status']).size()\
                        .to_frame('Count')

Count
Rep           Status          
Cedric Moss   pending        1
              presented      2
              won            1
Craig Booker  declined       1
              pending        1
              presented      2
Daniel Hilton pending        1
              presented      1
              won            1
John Smith    declined       1
              pending        1
Wendy Yule    declined       1
              presented      1
              won            2

```
select Rep, Status, count(*) from sales group by Rep, Status order by Rep, count(*) desc;
 ```

In [42]:
sales[['Rep', 'Status']].groupby(['Rep', 'Status']).size()\
                        .to_frame('Count').sort_values(['Rep', 'Count'],
                                                       ascending=[True, False])

Count
Rep           Status          
Cedric Moss   presented      2
              pending        1
              won            1
Craig Booker  presented      2
              declined       1
              pending        1
Daniel Hilton pending        1
              presented      1
              won            1
John Smith    declined       1
              pending        1
Wendy Yule    won            2
              declined       1
              presented      1

```
select sum(Quantity*Price) from sales where Status = 'won';
```

In [44]:
sales['Total'] = sales.Quantity*sales.Price

In [51]:
sales.query('Status == "won"').Total.sum()

781000

```
select Rep, Status, sum(Quantity*Price) from sales group by Rep, Status;
```

In [47]:
sales.pivot_table(index=['Rep', 'Status'], values='Total', aggfunc=np.sum)

Total
Rep           Status           
Cedric Moss   pending      5000
              presented   40000
              won        130000
Craig Booker  declined    35000
              pending     10000
              presented   40000
Daniel Hilton pending     80000
              presented   10000
              won        130000
John Smith    declined    35000
              pending     10000
Wendy Yule    declined   130000
              presented   10000
              won        521000